# Dominant Sequences
- Find dominant strains using edit distance

In [1]:
import os 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from domseq import DomSeq, save_model, load_model

## Downloading Data
**Sources: [GISAID](https://platform.epicov.org/epi3/cfrontend#586f5f) and [NCBI](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Protein&HostLineage_ss=Homo%20sapiens%20(human),%20taxid:9606&LabHost_s=include&ProtNames_ss=hemagglutinin&CollectionDate_dr=2003-11-01T00:00:00.00Z%20TO%202004-05-01T23:59:59.00Z&SLen_i=550%20TO%20566&VirusLineage_ss=H1N1%20subtype,%20taxid:114727)**
1. Download amino acid data
    - Host: Human
    - Flu Season example: 
        - Northern strains from 11/01/2002 - 5/01/2003
        - Southern strains from 05/01/2003 - 11/01/2003
        - Flu season dates from [CDC](https://www.cdc.gov/flu/school-business/travelersfacts.htm) and [WHO](https://www.who.int/teams/global-influenza-programme/vaccines/who-recommendations/recommendations-for-influenza-vaccine-composition-archive)
    - Segment: HA (4) and NA (6)
2. File names for raw data: HEMISPHERE_SEQUENCE_SEGMENT_SEASON
    - HEMISPHERE: "north" or "south"
    - SEQUENCE: "h1n1" or "h3n2"
    - SEGMENT: "ha" or "na"
    - SEASON: (ex. 02_03 for north 2002-2003, 02 for south 2002)
    - SEQUENCE LENGTH (NCBI)
        - HA: min = 550, max = 570
        - NA: min = 450, max = 470
3. Do this for each **north** flu season from 2002-2003 to 2021-2022 (21 seasons)
3. Do this for each **south** flu season from 2003 to 2022 (21 seasons)
5. In merge NCBI and GISAID data and save to `raw_data/merged/`
    - Some seasons will have no strains from a particular database
    - In each year record how many strains come from NCBI and GISAID

In [2]:
NCBI_DIR = 'raw_data/ncbi/'
GISAID_DIR = 'raw_data/gisaid/'
DATA_DIR = 'raw_data/merged/'
OUT_DIR = 'results/dominant_sequences/'

FILES = ['north_h1n1_ha', 'north_h1n1_na', 'north_h3n2_ha', 'north_h3n2_na',
         'south_h1n1_ha', 'south_h1n1_na', 'south_h3n2_ha', 'south_h3n2_na']

NORTH_YEARS = []
for i in np.arange(1, 22):
    YEAR = ''
    if i < 10:
        YEAR += '0' + str(i)
    else:
        YEAR += (str(i))
    if i + 1 < 10:
        YEAR += '_0' + str(i + 1)
    else:
        YEAR += '_' + str(i + 1)
    NORTH_YEARS.append(YEAR)
        
SOUTH_YEARS = []
for i in np.arange(2, 23):
    if i < 10:
        SOUTH_YEARS.append('0' + str(i))
    else:
        SOUTH_YEARS.append(str(i))

NA_TRUNC = 468 # 2 less than official length of 470
HA_TRUNC = 565 # 2 less than official length of 567

In [3]:
# returns merged sequence dataframe and number of sequences from each
def load_ncbi_gisaid(domseq, NCBI_FILE, GISAID_FILE):
    seq_df = pd.DataFrame({'acc':[],'name':[],'sequence':[]})
    num_ncbi = 0
    num_gisaid = 0
    if os.path.isfile(NCBI_FILE):
        seq_df_ncbi = domseq.load_data(NCBI_FILE)
        seq_df = seq_df.append(seq_df_ncbi)
        num_ncbi += len(seq_df_ncbi)
    if os.path.isfile(GISAID_FILE):
        seq_df_gisaid = domseq.load_data(GISAID_FILE)
        seq_df = seq_df.append(seq_df_gisaid)
        num_gisaid += len(seq_df_gisaid)
    return seq_df, num_ncbi, num_gisaid

In [4]:
num_seqs_north = pd.DataFrame(index=NORTH_YEARS)

for FILE in FILES[:4]:
    TRUNC = NA_TRUNC
    if FILE[-2:] == 'ha':
        TRUNC = HA_TRUNC
    # initialize the DomSeq
    domseq = DomSeq(seq_trunc_length=TRUNC, random_state=42)
    # store number of sequences available
    num_seqs_ncbi = []
    num_seqs_gisaid = []
    
    for YEAR in NORTH_YEARS:
        # load fasta data
        NAME = FILE+'_'+YEAR
        NCBI_FILE = NCBI_DIR+FILE+'/'+NAME+'.fasta'
        GISAID_FILE = GISAID_DIR+FILE+'/'+NAME+'.fasta'
        seq_df, num_ncbi, num_gisaid = load_ncbi_gisaid(domseq, NCBI_FILE, GISAID_FILE)
        seq_df.to_csv(DATA_DIR+FILE+'/'+NAME+'.csv', index=False)
        num_seqs_ncbi.append(num_ncbi)
        num_seqs_gisaid.append(num_gisaid)
    num_seqs_north['ncbi_'+FILE[-7:]] = num_seqs_ncbi
    num_seqs_north['gisaid_'+FILE[-7:]] = num_seqs_gisaid
    num_seqs_north['total_'+FILE[-7:]] = np.array(num_seqs_ncbi) + np.array(num_seqs_gisaid)
    
num_seqs_north.to_csv('results/num_seqs_north.csv')  
num_seqs_north

,ncbi_h1n1_ha,gisaid_h1n1_ha,total_h1n1_ha,ncbi_h1n1_na,gisaid_h1n1_na,total_h1n1_na,ncbi_h3n2_ha,gisaid_h3n2_ha,total_h3n2_ha,ncbi_h3n2_na,gisaid_h3n2_na,total_h3n2_na
01_02,33,6,39,33,13,46,137,140,277,131,124,255
02_03,59,22,81,36,24,60,79,56,135,84,52,136
03_04,18,5,23,22,12,34,264,223,487,275,223,498
04_05,45,6,51,44,18,62,201,171,372,213,168,381
05_06,116,44,160,74,61,135,142,123,265,254,219,473
06_07,555,536,1091,437,481,918,303,351,654,264,326,590
07_08,434,538,972,430,761,1191,474,472,946,333,446,779
08_09,1217,1250,2467,1247,1545,2792,492,569,1061,378,720,1098
09_10,2613,3406,6019,2213,2779,4992,143,207,350,64,162,226
10_11,1421,1773,3194,1166,1748,2914,764,999,1763,678,1062,1740


In [5]:
num_seqs_south = pd.DataFrame(index=SOUTH_YEARS)

for FILE in FILES[4:]:
    TRUNC = NA_TRUNC
    if FILE[-2:] == 'ha':
        TRUNC = HA_TRUNC
    # initialize the DomSeq
    domseq = DomSeq(seq_trunc_length=TRUNC, random_state=42)
    # store number of sequences available
    num_seqs_ncbi = []
    num_seqs_gisaid = []
       
    for YEAR in SOUTH_YEARS:
        # load fasta data
        NAME = FILE+'_'+YEAR
        NCBI_FILE = NCBI_DIR+FILE+'/'+NAME+'.fasta'
        GISAID_FILE = GISAID_DIR+FILE+'/'+NAME+'.fasta'
        seq_df, num_ncbi, num_gisaid = load_ncbi_gisaid(domseq, NCBI_FILE, GISAID_FILE)
        seq_df.to_csv(DATA_DIR+FILE+'/'+NAME+'.csv', index=False)
        num_seqs_ncbi.append(num_ncbi)
        num_seqs_gisaid.append(num_gisaid)
    num_seqs_south['ncbi_'+FILE] = num_seqs_ncbi
    num_seqs_south['gisaid_'+FILE] = num_seqs_gisaid
    num_seqs_south['total_'+FILE[-7:]] = np.array(num_seqs_ncbi) + np.array(num_seqs_gisaid)
    
num_seqs_south.to_csv('results/num_seqs_south.csv')
num_seqs_south

,ncbi_south_h1n1_ha,gisaid_south_h1n1_ha,total_h1n1_ha,ncbi_south_h1n1_na,gisaid_south_h1n1_na,total_h1n1_na,ncbi_south_h3n2_ha,gisaid_south_h3n2_ha,total_h3n2_ha,ncbi_south_h3n2_na,gisaid_south_h3n2_na,total_h3n2_na
02,8,0,8,11,1,12,125,142,267,125,119,244
03,23,12,35,26,19,45,205,176,381,210,187,397
04,6,4,10,8,42,50,219,201,420,228,212,440
05,67,32,99,51,45,96,209,180,389,229,205,434
06,73,39,112,45,68,113,12,15,27,32,41,73
07,138,188,326,93,186,279,201,290,491,141,237,378
08,209,328,537,262,448,710,163,202,365,95,232,327
09,4977,6043,11020,4059,4463,8522,503,754,1257,380,723,1103
10,577,870,1447,488,667,1155,587,854,1441,436,753,1189
11,217,453,670,198,375,573,449,782,1231,450,763,1213


## Dominant Sequence
Levenshtein Centroid: $$\widehat{x}^{dom} = argmin_{x\in P^t} \sum_{y \in P^t} \theta(x,y)$$
- Where $P^t$ is the sequence population at time $t$.
- $\theta(x,y)$ is the edit distance (Levenshtein) between x and y
- Do this for north starting at 2002-2003 and south starting at 2003
    - North 2001-2002 and south 2002 data used for Emergenet prediction

### NORTH

In [ ]:
for FILE in FILES[:4]:
    TRUNC = NA_TRUNC
    if FILE[-2:] == 'ha':
        TRUNC = HA_TRUNC
    # initialize the DomSeq
    domseq = DomSeq(seq_trunc_length=TRUNC, random_state=42)
    
    for YEAR in NORTH_YEARS[1:]:
        # load fasta data
        NAME = FILE+'_'+YEAR
        seq_df = pd.read_csv(DATA_DIR+FILE+'/'+NAME+'.csv')
        # compute dominant sequences
        dom_id, dom_seq, dom_df = domseq.compute_domseq(seq_df, sample_size=1000)
        # save to csv
        dom_df.to_csv(OUT_DIR+FILE+'/'+NAME+'.csv', index=False)

### SOUTH

In [ ]:
for FILE in FILES[4:]:
    TRUNC = NA_TRUNC
    if FILE[-2:] == 'ha':
        TRUNC = HA_TRUNC
    # initialize the DomSeq
    domseq = DomSeq(seq_trunc_length=TRUNC, random_state=42)
    
    for YEAR in SOUTH_YEARS[1:]:
        # load fasta data
        NAME = FILE+'_'+YEAR
        seq_df = pd.read_csv(DATA_DIR+FILE+'/'+NAME+'.csv')
        # compute dominant sequences
        dom_id, dom_seq, dom_df = domseq.compute_domseq(seq_df, sample_size=1000)
        # save to csv
        dom_df.to_csv(OUT_DIR+FILE+'/'+NAME+'.csv', index=False)

## Distance to Centroid
- These are the 10 dominant strains we compare our predictions against
- For each of the 10 computed dominant strains in each season, find the average Levenshtein distance all other strains in that season
    - Average these 10 averages

In [5]:
dom_dists_north = pd.DataFrame({'north_season':NORTH_YEARS[1:]})
for FILE in FILES[:4]:
    dists = []
    for n in range(1, 21):
        NAME = FILE + '_' + NORTH_YEARS[n]
        DATA_DIR = OUT_DIR + FILE + '/' + NAME + '.csv'
        dom_df = pd.read_csv(DATA_DIR)['total_edit_dist']
        top_seqs = dom_df[:10]
        dists.append(sum(top_seqs)/(10*len(dom_df)))
    dom_dists_north[FILE] = dists
        
dom_dists_south = pd.DataFrame({'south_season':SOUTH_YEARS[1:]})
for FILE in FILES[4:]:
    dists = []
    for n in range(1, 21):
        NAME = FILE + '_' + SOUTH_YEARS[n]
        DATA_DIR = OUT_DIR + FILE + '/' + NAME + '.csv'
        dom_df = pd.read_csv(DATA_DIR)['total_edit_dist']
        top_seqs = dom_df[:10]
        dists.append(sum(top_seqs)/(10*len(dom_df)))
    dom_dists_south[FILE] = dists
        
dom_dists = dom_dists_north.join(dom_dists_south, how='outer')
dom_dists.to_csv(OUT_DIR + 'dom_dists.csv', index=False)
dom_dists

,north_season,north_h1n1_ha,north_h1n1_na,north_h3n2_ha,north_h3n2_na,south_season,south_h1n1_ha,south_h1n1_na,south_h3n2_ha,south_h3n2_na
0,02_03,2.135802,2.333333,6.508148,19.073529,03,1.685714,1.133333,2.023622,4.372796
1,03_04,1.695652,14.852941,3.036961,10.321285,04,4.560000,11.800000,2.497619,3.713636
2,04_05,2.578431,23.548387,2.930108,15.144357,05,7.230303,41.843750,3.786632,4.792627
3,05_06,8.074375,4.229630,3.966038,4.078224,06,6.808929,15.290265,5.196296,6.100000
4,06_07,5.211000,7.259259,5.044343,4.664407,07,15.432515,9.888889,4.391039,5.301587
5,07_08,3.537037,11.002000,7.352008,7.592426,08,7.385475,9.470282,13.375342,1.929664
6,08_09,192.089000,47.834000,6.549000,2.489000,09,9.755000,5.827000,3.349600,2.430000
7,09_10,5.267000,7.073000,5.391429,2.641593,10,7.170000,2.983000,5.178000,2.194000
8,10_11,4.953600,2.730000,5.171000,2.710000,11,5.140299,2.860384,6.486000,3.525000
9,11_12,5.192033,2.861005,6.403600,3.837000,12,6.914335,3.457249,13.840500,6.578000


## Unique Sequences Per Season
- Count number of unique sequences per season and their frequencies, within the sample of 1000

In [64]:
unique_df = pd.DataFrame()
for FILE in FILES:
    YEARS = NORTH_YEARS
    unique_df['north_season'] = NORTH_YEARS[1:]
    if FILE[:5] == 'south':
        YEARS = SOUTH_YEARS
        unique_df['south_season'] = SOUTH_YEARS[1:]
    vals = []
    for n in range(1, 21):
        unique_seqs = {}
        NAME = FILE + '_' + YEARS[n]
        DATA_DIR = OUT_DIR + FILE + '/' + NAME + '.csv'
        dom_df = pd.read_csv(DATA_DIR)['sequence']
        for seq in dom_df:
            if len(seq) != HA_TRUNC and len(seq) != NA_TRUNC:
                print(-1)
            if seq not in unique_seqs:
                unique_seqs[seq] = 1
            else:
                unique_seqs[seq] += 1
        if max(unique_seqs.values()) in list(unique_seqs.values())[:10]:
            vals.append(1)
        else:
            vals.append(0)
            
        # save to new dataframe, in order of frequency rather than distance to other seqs
        unique_seqs = dict(sorted(unique_seqs.items(), key=lambda item: item[1], reverse=True))
        df = pd.DataFrame({'name':unique_seqs.keys(), 'value':unique_seqs.values()})
        df = df.rename(columns={'name':'sequence','value':'frequency'})
        
    unique_df[FILE] = vals
unique_df

,north_season,north_h1n1_ha,north_h1n1_na,north_h3n2_ha,north_h3n2_na,south_season,south_h1n1_ha,south_h1n1_na,south_h3n2_ha,south_h3n2_na
0,02_03,1,1,1,1,03,1,1,1,1
1,03_04,1,1,1,1,04,1,1,1,1
2,04_05,1,1,1,1,05,1,1,1,1
3,05_06,1,1,1,1,06,0,1,1,0
4,06_07,1,1,1,1,07,1,1,1,1
5,07_08,1,1,1,1,08,1,0,1,1
6,08_09,0,0,1,1,09,1,1,1,1
7,09_10,1,1,0,1,10,1,1,0,1
8,10_11,1,1,1,1,11,1,1,1,1
9,11_12,1,1,1,1,12,0,1,1,1


In [65]:
# number/proportion of seasons where most frequent strain is not in top 10 dominant strains
unique_df = unique_df.drop(columns={'north_season', 'south_season'})
print(160-unique_df.sum().sum())
print((160-unique_df.sum().sum())/(len(unique_df)*8))

20
0.125
